In [ ]:
# to install python packages at runtime use ! as prefix
!pip install rdflib
!pip install deep_translator
from rdflib import BNode, URIRef, Literal, Graph, Namespace
from rdflib.collection import Collection
from rdflib.util import guess_format
from rdflib.namespace import RDF, XSD, RDFS, OWL, SKOS, DCTERMS, NamespaceManager
from rdflib.plugins.sparql import prepareQuery
from typing import List
from re import sub
from deep_translator import GoogleTranslator
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Snake Chase - your_term
def snake_case(s):
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    s.replace('-', ' '))).split()).lower()

# Camel Chase - yourTerm
def lower_camel_case(s):
  #print(s)
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

# Pascal Chase -  YourTerm
def upper_camel_case(s):
  #print(s)
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return s

def strip_special_chars(s):
    #return sub('[^A-Za-z0-9]+', ' ', s)
    return sub('\W+',' ', s )






In [ ]:
# add namespaces if needed
bfo2020_url='http://purl.obolibrary.org/obo/bfo/2020/bfo.owl'
BFO = Namespace(bfo2020_url+"/")
OBO = Namespace('http://purl.obolibrary.org/obo/')


In [ ]:
# load an ontology from the web
g = Graph()
g.parse("https://kupferdigital.gitlab.io/ontologies/kdo/ontology.ttl")

In [ ]:
# get all subclasses of an entity
sub_classes = prepareQuery("SELECT ?entity WHERE {?entity rdfs:subClassOf* ?parent}")
def get_all_sub_classes(superclass: URIRef, ontology: Graph) -> List[URIRef]:
    """Gets all subclasses of a given class.

    Args:
        superclass (URIRef): Rdflib URIRef of the superclass

    Returns:
        List[URIRef]: List of all subclasses
    """
    # parse template and add mapping results
    results = list(
        ontology.query(
            sub_classes,
            initBindings={"parent": superclass},
            # initNs={'cco': CCO, 'mseo': MSEO},
        ),
    )
    # print(list(ontology[ : RDFS.subClassOf]))
    classes = [result[0] for result in results]
    logger.debug("Found following subclasses of {}: {}".format(superclass, classes))
    logger.debug("Found {} subclasses of {}".format(len(classes),superclass))
    return classes


In [ ]:
# simple query
i=0
for s,p,o in g.triples((None,None,None)):
  print(s,p,o)
  i+=1
  if i>20:
    break


In [ ]:
# translate all labels with Google Translator

all_labels = prepareQuery("SELECT ?entity ?label WHERE {?entity rdfs:label ?label}")

def translate_labels_google(g: Graph, language: str='de'):
    translator=GoogleTranslator(source='auto', target=language)
    res=dict()
    labels=g.query(all_labels)
    for thing, label in labels:
        if thing not in res.keys():
            res[thing]={}
        res[thing][label.language]=label
    i=0
    for thing, labels in res.items():
        if not all(lang in labels.keys() for lang in ("en","de")):
            #entitys with only one label
            #print(thing,labels)
            #translate and add triple
            if labels.get('en',None):
                label_de=Literal(translator.translate(labels['en']),lang=language)
                logging.info('adding german label {} for entity {}'.format(label_de, thing))
                g.add((thing,RDFS.label,label_de))
                #add curation status - requires discussion
                g.add((thing,OBO.IAO_0000114,OBO.IAO_0000428))
                i+=1
    logging.info("translated {} labels to [{}]".format(i,language))
    return g

onto=translate_labels_google(g,language='de')

